In [2]:
import numpy as np
import multiprocessing
import ctypes

In [3]:
A = np.array(
    [
        [[1, 2], [4, 5]],
        [[0, 2], [1, 6]],
    ]
    ,dtype=np.float32
)

In [10]:
A = A.astype(np.float64)
A.dtype

dtype('float64')

In [11]:
A

array([[[1., 2.],
        [4., 5.]],

       [[0., 2.],
        [1., 6.]]])

In [7]:
np.finfo(np.float64)

finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)

In [29]:
shared_array_base = multiprocessing.Array(ctypes.c_double, 10 * 10)
a_ilj_final = np.ctypeslib.as_array(shared_array_base.get_obj())
a_ilj_final = a_ilj_final.reshape(10, 10)


# -- edited 2015-05-01: the assert check below checks the wrong thing
#   with recent versions of Numpy/multiprocessing. That no copy is made
#   is indicated by the fact that the program prints the output shown below.
## No copy was made
##assert shared_array.base.base is shared_array_base.get_obj()


def my_func(i, def_param=a_ilj_final, param=A):
    a_ilj_final[i, :] = i
    print(id(A))


if __name__ == "__main__":
    pool = multiprocessing.Pool(processes=4)
    pool.map(my_func, range(10))

    print(a_ilj_final)

140650840592080 140650840592080 140650840592080 140650840592080 



140650840592080 140650840592080 140650840592080 140650840592080 


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [9. 9. 9. 9. 9. 9. 9. 9. 9. 9.]]

140650840592080 140650840592080 



In [48]:
from tqdm import tqdm
from multiprocessing import Pool
from functools import partial


class DefaultPfe:
    def __init__(self, T=1, sigma_shift=0) -> None:
        self.T = T
        self.sigma_shift = sigma_shift

    @staticmethod
    def per_dim_computation(k, mu, sigma, z, a_ilj_final):
        a_ilj = (
            z[:, :, np.newaxis, k] - mu[np.newaxis, np.newaxis, :, k]
        ) ** 2 / sigma[np.newaxis, np.newaxis, :, k] + np.log(sigma)[
            np.newaxis, np.newaxis, :, k
        ]
        np.add(a_ilj_final, a_ilj, out=a_ilj_final)

    def __call__(self, mu, sigma, z):

        a_ilj_final = np.zeros(shape=z.shape[:-1] + mu.shape[:-1])  # placeholder
        shared_array_base = multiprocessing.Array(
            ctypes.c_double, int(np.multiply.reduce(list(z.shape[:-1] + mu.shape[:-1])))
        )
        a_ilj_final = np.ctypeslib.as_array(shared_array_base.get_obj())
        a_ilj_final = a_ilj_final.reshape(z.shape[:-1] + mu.shape[:-1])

        num_dims = mu.shape[1]
        sigma = sigma + self.sigma_shift
        with Pool(4) as pool_fc:
            for _ in tqdm(
                pool_fc.imap_unordered(
                    partial(
                        self.per_dim_computation,
                        mu=mu,
                        sigma=sigma,
                        z=z,
                        a_ilj_final=a_ilj_final,
                    ),
                    range(num_dims),
                ),
                total=len(range(num_dims)),
            ):
                pass

        return -0.5 * a_ilj_final / self.T


fpe = DefaultPfe(1, 0)
a = fpe(A, A, A)

  0%|          | 0/2 [00:00<?, ?it/s]/tmp/ipykernel_32159/3613483182.py:12: RuntimeWarning: divide by zero encountered in log
  ) ** 2 / sigma[np.newaxis, np.newaxis, :, k] + np.log(sigma)[
/tmp/ipykernel_32159/3613483182.py:10: RuntimeWarning: divide by zero encountered in true_divide
  a_ilj = (
/tmp/ipykernel_32159/3613483182.py:12: RuntimeWarning: divide by zero encountered in log
  ) ** 2 / sigma[np.newaxis, np.newaxis, :, k] + np.log(sigma)[
/tmp/ipykernel_32159/3613483182.py:10: RuntimeWarning: invalid value encountered in add
  a_ilj = (
100%|██████████| 2/2 [00:00<00:00, 141.64it/s]


In [49]:
a.shape

(2, 2, 2, 2)

In [33]:
np.multiply(z.shape[:-1] + mu.shape[:-1])

NameError: name 'z' is not defined